In [1]:
import pandas as pd
import numpy as np


data = pd.read_csv("fer2013.csv")
data

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [2]:
data['Usage'].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [3]:
X_train = []
y_train = []
X_test = []
y_test = []
#X_val = []
#y_val = []

for index,row in data.iterrows():
  k = row['pixels'].split(" ")
  if row['Usage'] == 'Training':
    X_train.append(np.array(k))
    y_train.append(row['emotion'])

  #if row['Usage'] == 'PrivateTest':
  #  X_val.append(np.array(k))
  #  y_val.append(row['emotion'])  
    
  elif row['Usage'] == 'PublicTest':
    X_test.append(np.array(k))
    y_test.append(row['emotion'])

In [4]:
import keras
from keras.utils import np_utils
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
#X_val = np.array(X_val)
#y_val = np.array(y_val)

X_train_new = X_train.reshape(X_train.shape[0],48,48,1)
X_test_new = X_test.reshape(X_test.shape[0],48,48,1)
#X_val_new = X_val.reshape(X_val.shape[0],48,48,1)

y_train_new = np_utils.to_categorical(y_train,num_classes=10)
y_test_new = np_utils.to_categorical(y_test,num_classes=10)
#y_val_new = np_utils.to_categorical(y_val,num_classes=10)

In [5]:
X_train_new = X_train_new.astype('float32')/255
X_test_new = X_test_new.astype('float32')/255
#X_val_new = X_val_new.astype('float32')/255

In [6]:
import keras
from keras.models import *
from keras.layers import *
from keras.models import Sequential
from keras.layers.core import Flatten,Dense,Dropout
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import SGD
pic_width = 48
pic_height= 48
n_channels = 1

#280-5+1
#300-5+1

#(280-5+1)/2 
#(300-5+1)/2

#(280-5+1)/3 = 276/3 = 92
#ceiling((296)/3 = 98+1


# If it a colored image ..the filter will automaticall have 3  channels

model = Sequential()
# Input layer in CNN should have same shape as original images ( no flattening required)
model.add(InputLayer(input_shape=(pic_width,pic_height,n_channels)))
# Adding a conv layer
model.add(Conv2D(32,(5,5),activation = 'relu',strides = (1,1)))   # 32 = no . of filters present in this conv layer; 
model.add(MaxPooling2D(2,2))                                                                  #(5 x 5) = size of the filter
                                                                  #strides = (1,1) = hprozonatl 1 slide , vertical 1 slide
# Adding a second conv layer (optional)
model.add(Conv2D(64,(5,5),activation = 'relu',strides = (1,1))) 
model.add(MaxPooling2D(2,2))
# Flattening of the conv layer output
model.add(Flatten())
# MLP
model.add(Dense(100,activation = 'relu')) # relu - works best
model.add(Dense(10,activation = 'softmax'))   # Why sigmoid -- binary --- cat vs dog

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 44, 44, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               518500    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [7]:
X_train.shape

(28709, 2304)

In [8]:
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
history=model.fit(X_train_new,y_train_new,batch_size=32,epochs=30,validation_data=(X_test_new,y_test_new))

Epoch 1/30
898/898 [==============================] - 355s 395ms/step - loss: 1.8232 - accuracy: 0.2501 - val_loss: 1.7926 - val_accuracy: 0.3037
Epoch 2/30
898/898 [==============================] - 350s 389ms/step - loss: 1.7375 - accuracy: 0.3068 - val_loss: 1.8282 - val_accuracy: 0.2499
Epoch 3/30
898/898 [==============================] - 370s 412ms/step - loss: 1.6462 - accuracy: 0.3656 - val_loss: 1.6273 - val_accuracy: 0.3870
Epoch 4/30
898/898 [==============================] - 354s 394ms/step - loss: 1.5884 - accuracy: 0.3903 - val_loss: 1.6611 - val_accuracy: 0.3313
Epoch 5/30
898/898 [==============================] - 363s 404ms/step - loss: 1.5453 - accuracy: 0.4072 - val_loss: 1.8016 - val_accuracy: 0.3302
Epoch 6/30
898/898 [==============================] - 356s 396ms/step - loss: 1.5064 - accuracy: 0.4242 - val_loss: 1.5670 - val_accuracy: 0.3912
Epoch 7/30
898/898 [==============================] - 367s 409ms/step - loss: 1.4711 - accuracy: 0.4393 - val_loss: 1.5264 -

In [10]:
model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)
  model.save_weights("model.h5")

In [1]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [2]:
model = model_from_json(open("model.json","r").read())
model.load_weights('model.h5')

In [3]:
face_cl = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(0)

In [4]:
while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_cl.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>